In [1]:
import pandas as pd
import re
import itertools
import networkx as nx

In [2]:
df = pd.read_csv('./data/film_festivals.csv', keep_default_na=False)

In [3]:
df_network = df[~(df['Director'].isin(['N/A', '']) & df['Writer'].isin(['N/A', '']) & df['Actors'].isin(['N/A', '']))][['Director', 'Writer', 'Actors']]
df_network['Writer'] = df_network['Writer'].apply(lambda row: re.sub(' \(.*?\)', '', row))
df_network.head()

,Director,Writer,Actors
0,Youssef Chahine,"Youssef Chahine, Yusri Nasrullah","Michel Piccoli, Mohsen Mohieddin, Patrice Chér..."
1,Alan Parker,"William Wharton, Sandy Kroopf, Jack Behr","Matthew Modine, Nicolas Cage, John Harkins, Sa..."
2,Ray Lawrence,"Ray Lawrence, Peter Carey, Peter Carey","Barry Otto, Lynette Curran, Helen Jones, Miles..."
4,Dusan Makavejev,"Frank Moorhouse, Denny Lawrence, Frank Moorhouse","Eric Roberts, Greta Scacchi, Bill Kerr, Chris ..."
5,István Szabó,"István Szabó, Péter Dobai, John Osborne","Klaus Maria Brandauer, Hans Christian Blech, A..."


In [4]:
directors = df_network.iloc[0]['Director'].split(', ')
directors = [(director, 'director') for director in directors]
writers = df_network.iloc[0]['Writer'].split(', ')
writers = [(writer, 'writer') for writer in writers]
actors = df_network.iloc[0]['Actors'].split(', ')
actors = [(actor, 'actor') for actor in actors]
people = directors
people.extend(actors) # the other of the extends determines the order of priority: director -> actor -> writer for people who have multiple roles
people.extend(writers)

In [5]:
name = [person[0] for person in people]
role = [person[1] for person in people]
nodelist = pd.DataFrame({'Name': name,
                         'Role': role})
nodelist = nodelist.drop_duplicates(subset='Name').reset_index(drop=True)
nodelist

,Name,Role
0,Youssef Chahine,director
1,Michel Piccoli,actor
2,Mohsen Mohieddin,actor
3,Patrice Chéreau,actor
4,Mohamad Atef,actor
5,Yusri Nasrullah,writer


In [6]:
edges = [combo for combo in itertools.combinations(nodelist['Name'], 2)]
edgelist = pd.DataFrame(edges, columns=['Source', 'Target'])
edgelist

,Source,Target
0,Youssef Chahine,Michel Piccoli
1,Youssef Chahine,Mohsen Mohieddin
2,Youssef Chahine,Patrice Chéreau
3,Youssef Chahine,Mohamad Atef
4,Youssef Chahine,Yusri Nasrullah
5,Michel Piccoli,Mohsen Mohieddin
6,Michel Piccoli,Patrice Chéreau
7,Michel Piccoli,Mohamad Atef
8,Michel Piccoli,Yusri Nasrullah
9,Mohsen Mohieddin,Patrice Chéreau
